In [1]:
import numpy as np
import pandas as pd
import os
import cv2
import sklearn
import itertools
import keras
import time
from keras import applications
from keras.models import Sequential, Model
from keras.layers import (Lambda,GlobalAveragePooling1D, BatchNormalization, Conv1D, Dense, Input, TimeDistributed, Activation, Bidirectional, SimpleRNN, GRU, LSTM)
from keras.models import load_model
import keras.backend.tensorflow_backend as KTF
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.regularizers import l2,l1
from keras import optimizers

import random
from random import shuffle


#from utils.model_utils import *

os.getcwd()

Using TensorFlow backend.


'/home/te_pana/Pictures/LID'

In [2]:
IMAGE_HEIGHT = 256
IMAGE_WIDTH = 860
IMAGE_CHANNEL = 1

num_classes = 3

TRAIN_BATCH_SIZE = 32
VALIDATION_BATCH_SIZE = 32
NUM_TRAIN_EPOCHS = 480

TRAINED_MODEL_FILE_NAME = "path-to-model/Lang_classification-2-epoch-{epoch:2d}-val_acc-{val_acc:.2f}-val_loss-{val_loss:.2f}.h5"

In [3]:
train_path = 'path-to-training-dataset'
val_path = 'path-to-testing-dataset'
# train_path = '/home/te_pana/Pictures/LID/Validation_spc/'
train_samples = list()
validation_samples = list()

for root, directories, files in os.walk(train_path):
    for filename in files:
        train_samples.append(filename)

shuffle(train_samples)
shuffle(train_samples)
shuffle(train_samples)



for root, directories, files in os.walk(val_path):
    for filename in files:
        validation_samples.append(filename)

# train, test = train_test_split(samples, train_size = 0.96)
# for root, directories, files in os.walk(val_path):
#     for filename in files:
#         validation_samples.append(filename)

print("Before Shuffiling Train  " + train_samples[20])
shuffle(train_samples)
print("After Shuffiling  Train  " + train_samples[20])

print("Before Shuffiling Val    " + validation_samples[20])
shuffle(validation_samples)
shuffle(validation_samples)
shuffle(validation_samples)
print("After Shuffiling  Val    " + validation_samples[20])

Before Shuffiling Train  0_Dudley South - General Election Declaration_Chunk_100000__Pitch_1_2_Chunk_20000_.jpg
After Shuffiling  Train  2_org_财经追击 News Report on Lunar New Year and Population 2013 in Mandarin-MbvA1J1OHjw_Chunk_450000_Chunk_10000_.jpg
Before Shuffiling Val    1_Japanese Movie 'ShodouGirlsーBlueBlueSky'　(English Subtitled)-5GP7MwkKcF0Chunk_578_.jpg
After Shuffiling  Val    1_Japanese Movie 'ShodouGirlsーBlueBlueSky'　(English Subtitled)-5GP7MwkKcF0Chunk_336_.jpg


In [4]:
print( len(validation_samples) , len(train_samples))

2345 125712


In [5]:
def generator(samples, path, batch_size):
    shuffle(samples)
    while(1):
        
        for x in range(0,len(samples),batch_size):
            Input_Image1 = []
            Input_label1 = []     
            
            for Image in samples[x:x+batch_size]:
                img = cv2.imread(path+Image, 0)
                img = img.reshape(IMAGE_CHANNEL, IMAGE_WIDTH, IMAGE_HEIGHT)
                label = Image.split("_")[0]
                label = np.eye(num_classes)[int(label)]
                Input_Image1.append(img[0])
                Input_label1.append(label)
                
            X_train = np.array(Input_Image1)
            y_train = np.array(Input_label1)
            
            yield sklearn.utils.shuffle(X_train, y_train)

In [6]:
train_generator = generator(samples = train_samples, path = train_path, batch_size = TRAIN_BATCH_SIZE)
valid_generator = generator(samples = validation_samples, path = val_path, batch_size = VALIDATION_BATCH_SIZE)

In [3]:

def cnn_rnn(input_dim, filters, kernel_size, conv_stride,
    conv_border_mode, units, output_dim=3):
    """ Build a recurrent + convolutional network for speech 
    """
    # Main acoustic input
    input_data = Input(name='the_input', shape=(IMAGE_WIDTH, IMAGE_HEIGHT))
    input_data1 = Lambda(lambda x: x/255.0)(input_data)
    # Add convolutional layer
    conv_1d = Conv1D(filters, kernel_size, 
                     strides=conv_stride, 
                     padding=conv_border_mode,
                     activation='relu',
                     name='conv1d')(input_data1)
    # Add batch normalization
    bn_cnn = BatchNormalization(name='bn_conv_1d')(conv_1d)
    # Add a recurrent layer
    simp_rnn = GRU(units, activation='relu',
        return_sequences=True, implementation=2, name='rnn')(bn_cnn)
    # TODO: Add batch normalization
    bn_rnn = BatchNormalization(name='bn_rnn_1d')(simp_rnn)
    # TODO: Add a TimeDistributed(Dense(output_dim)) layer
    time_dense = TimeDistributed(Dense(output_dim))(bn_rnn)
    # Add softmax activation layer
#     a = Flatten()(time_dense)
    y_pred1 =  GlobalAveragePooling1D()(time_dense)
#     y_pred1 = Dense(3)(time_dense)
    y_pred = Activation('softmax', name='softmax')(y_pred1)
    

    # Specify the model
    model = Model(inputs=input_data, outputs=y_pred)
    model.output_length = lambda x: cnn_output_length(
        x, kernel_size, conv_border_mode, conv_stride)
    print(model.summary())
    return model

In [ ]:
def deep_rnn(input_dim, units, recur_layers, output_dim=29):
    """ Build a deep recurrent network for speech 
    """
    # Main acoustic input
    input_data = Input(name='the_input', shape=(None, input_dim))
    if recur_layers == 1:
        layer = LSTM(units, return_sequences=True, activation='relu')(input_data)
        layer = BatchNormalization(name='bt_rnn_1')(layer)
    else:
        layer = LSTM(units, return_sequences=True, activation='relu')(input_data)
        layer = BatchNormalization(name='bt_rnn_1')(layer)

        for i in range(recur_layers - 2):
            layer = LSTM(units, return_sequences=True, activation='relu')(layer)
            layer = BatchNormalization(name='bt_rnn_{}'.format(2+i))(layer)

        layer = LSTM(units, return_sequences=True, activation='relu')(layer)
        layer = BatchNormalization(name='bt_rnn_last_rnn')(layer)

    
    time_dense = TimeDistributed(Dense(output_dim))(layer)
    # Add softmax activation layer
    y_pred = Activation('softmax', name='softmax')(time_dense)
    # Specify the model
    model = Model(inputs=input_data, outputs=y_pred)
    model.output_length = lambda x: x
    print(model.summary())
return model

In [ ]:
def bidirectional_network(input_dim, units, output_dim=29):
    """ Build a bidirectional recurrent network for speech
    """
    # Main acoustic input
    input_data = Input(name='the_input', shape=(None, input_dim))
    bidir_rnn = Bidirectional(LSTM(units, return_sequences=True, activation='relu'), merge_mode='concat')(input_data)
    time_dense = TimeDistributed(Dense(output_dim))(bidir_rnn)
    # Add softmax activation layer
    y_pred = Activation('softmax', name='softmax')(time_dense)
    # Specify the model
    model = Model(inputs=input_data, outputs=y_pred)
    model.output_length = lambda x: x
    print(model.summary())
return model

In [4]:
model = cnn_rnn(input_dim=(860,256), # change to what ever model you want to use--> here cnn_rnn model is used
                        filters=200,
                        kernel_size=11, 
                        conv_stride=2,
                        conv_border_mode='valid',
                        units=200)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
the_input (InputLayer)       (None, 860, 256)          0         
_________________________________________________________________
lambda_1 (Lambda)            (None, 860, 256)          0         
_________________________________________________________________
conv1d (Conv1D)              (None, 425, 200)          563400    
_________________________________________________________________
bn_conv_1d (BatchNormalizati (None, 425, 200)          800       
_________________________________________________________________
rnn (GRU)                    (None, 425, 200)          240600    
_________________________________________________________________
bn_rnn_1d (BatchNormalizatio (None, 425, 200)          800       
_________________________________________________________________
time_distributed_1 (TimeDist (None, 425, 3)            603       
__________

In [ ]:
model.compile(loss='categorical_crossentropy',optimizer='adadelta',metrics=['accuracy'])

In [ ]:
checkpoint = keras.callbacks.ModelCheckpoint(filepath=TRAINED_MODEL_FILE_NAME)
model.fit_generator(train_generator, steps_per_epoch= (len(train_samples)/(TRAIN_BATCH_SIZE*4)),
            validation_data=valid_generator, validation_steps=len(validation_samples)/VALIDATION_BATCH_SIZE,
           epochs=NUM_TRAIN_EPOCHS, callbacks=[checkpoint])

Epoch 1/480
983/982 [==============================] - 1714s - loss: 0.9778 - acc: 0.5201 - val_loss: 2.6715 - val_acc: 0.2891
Epoch 2/480
243/982 [======>.......................] - ETA: 1272s - loss: 0.8560 - acc: 0.6128

In [ ]:
model.compile(loss='categorical_crossentropy',optimizer='adadelta',metrics=['accuracy']) 

In [ ]:
# model = load_model("/media/te_pana/Disk2/Others/LID/LID-2-epoch-21-val_acc-0.90-val_loss-0.47.h5")
checkpoint = keras.callbacks.ModelCheckpoint(filepath=TRAINED_MODEL_FILE_NAME)
model.fit_generator(train_generator, steps_per_epoch= (len(train)/(TRAIN_BATCH_SIZE)),
            validation_data=valid_generator, validation_steps=len(test)/VALIDATION_BATCH_SIZE,
           epochs=NUM_TRAIN_EPOCHS, callbacks = [checkpoint])

In [ ]:
Epoch 1/480
3010/3010 [==============================] - 3160s - loss: 0.6778 - acc: 0.7055 - val_loss: 0.8993 - val_acc: 0.6495
Epoch 2/480
3010/3010 [==============================] - 3156s - loss: 0.4718 - acc: 0.8129 - val_loss: 1.1379 - val_acc: 0.5430
Epoch 3/480
3010/3010 [==============================] - 3164s - loss: 0.3655 - acc: 0.8588 - val_loss: 0.6176 - val_acc: 0.7829
Epoch 4/480
3010/3010 [==============================] - 3161s - loss: 0.3017 - acc: 0.8848 - val_loss: 0.6285 - val_acc: 0.7509
Epoch 5/480
3010/3010 [==============================] - 3161s - loss: 0.2581 - acc: 0.9038 - val_loss: 0.3857 - val_acc: 0.8547
Epoch 6/480
3010/3010 [==============================] - 3161s - loss: 0.2208 - acc: 0.9187 - val_loss: 0.6774 - val_acc: 0.7361
Epoch 7/480
3010/3010 [==============================] - 3161s - loss: 0.1908 - acc: 0.9311 - val_loss: 0.2863 - val_acc: 0.8912
Epoch 8/480
3010/3010 [==============================] - 3161s - loss: 0.1683 - acc: 0.9401 - val_loss: 1.5436 - val_acc: 0.5864
Epoch 9/480
3010/3010 [==============================] - 3162s - loss: 0.1694 - acc: 0.9395 - val_loss: 0.8218 - val_acc: 0.7186
Epoch 10/480
3010/3010 [==============================] - 3164s - loss: 0.1469 - acc: 0.9476 - val_loss: 0.9280 - val_acc: 0.7257
Epoch 11/480
3010/3010 [==============================] - 3163s - loss: 0.1331 - acc: 0.9528 - val_loss: 1.3247 - val_acc: 0.5799
Epoch 12/480
3010/3010 [==============================] - 3163s - loss: 0.1274 - acc: 0.9550 - val_loss: 0.2721 - val_acc: 0.9030
Epoch 13/480
3010/3010 [==============================] - 3163s - loss: 0.1216 - acc: 0.9572 - val_loss: 0.7464 - val_acc: 0.7761
Epoch 14/480
3010/3010 [==============================] - 3162s - loss: 0.1116 - acc: 0.9609 - val_loss: 0.5203 - val_acc: 0.8250
Epoch 15/480
3010/3010 [==============================] - 3162s - loss: 0.0995 - acc: 0.9653 - val_loss: 0.6543 - val_acc: 0.8131
Epoch 16/480
3010/3010 [==============================] - 3162s - loss: 0.0904 - acc: 0.9683 - val_loss: 0.7863 - val_acc: 0.7837
Epoch 17/480
3010/3010 [==============================] - 3163s - loss: 0.0869 - acc: 0.9702 - val_loss: 0.3568 - val_acc: 0.8954
Epoch 18/480
3010/3010 [==============================] - 3162s - loss: 0.0839 - acc: 0.9714 - val_loss: 0.5483 - val_acc: 0.8587
Epoch 19/480
3010/3010 [==============================] - 3163s - loss: 0.0768 - acc: 0.9740 - val_loss: 0.3787 - val_acc: 0.8688
Epoch 20/480
3010/3010 [==============================] - 3162s - loss: 0.0695 - acc: 0.9760 - val_loss: 1.1309 - val_acc: 0.7591
Epoch 21/480
3010/3010 [==============================] - 3186s - loss: 0.0639 - acc: 0.9777 - val_loss: 0.6576 - val_acc: 0.8724
Epoch 22/480
3010/3010 [==============================] - 3350s - loss: 0.0624 - acc: 0.9787 - val_loss: 0.4688 - val_acc: 0.9022
Epoch 23/480
1781/3010 [================>.............] - ETA: 1272s - loss: 0.0599 - acc: 0.9793
            
            
Epoch 1/480
3010/3010 [==============================] - 3174s - loss: 0.0924 - acc: 0.9681 - val_loss: 1.0544 - val_acc: 0.7352
Epoch 2/480
3010/3010 [==============================] - 3180s - loss: 0.0783 - acc: 0.9726 - val_loss: 0.1891 - val_acc: 0.9294
Epoch 3/480
3010/3010 [==============================] - 3170s - loss: 0.0711 - acc: 0.9755 - val_loss: 0.2946 - val_acc: 0.9105
Epoch 4/480
3010/3010 [==============================] - 3170s - loss: 0.0665 - acc: 0.9770 - val_loss: 1.3096 - val_acc: 0.6007
Epoch 5/480
3010/3010 [==============================] - 3174s - loss: 0.0609 - acc: 0.9790 - val_loss: 0.3842 - val_acc: 0.8905
Epoch 6/480
3010/3010 [==============================] - 3207s - loss: 0.0621 - acc: 0.9791 - val_loss: 1.1011 - val_acc: 0.6243
Epoch 7/480
  59/3010 [..............................] - ETA: 3065s - loss: 0.0557 - acc: 0.9817

In [ ]:
Epoch 1/48
118/117 [==============================] - 66s - loss: 1.0640 - acc: 0.4167 - val_loss: 1.1140 - val_acc: 0.4846
Epoch 2/48
118/117 [==============================] - 65s - loss: 1.0275 - acc: 0.4655 - val_loss: 1.2309 - val_acc: 0.2469
Epoch 3/48
118/117 [==============================] - 64s - loss: 0.9708 - acc: 0.5231 - val_loss: 1.2130 - val_acc: 0.2809
Epoch 4/48
118/117 [==============================] - 64s - loss: 0.8971 - acc: 0.5935 - val_loss: 1.3103 - val_acc: 0.3457
Epoch 5/48
118/117 [==============================] - 64s - loss: 0.7958 - acc: 0.6585 - val_loss: 1.3534 - val_acc: 0.3025
Epoch 6/48
118/117 [==============================] - 64s - loss: 0.7033 - acc: 0.7182 - val_loss: 1.4585 - val_acc: 0.3025
Epoch 7/48
118/117 [==============================] - 65s - loss: 0.6196 - acc: 0.7500 - val_loss: 1.6011 - val_acc: 0.2994
Epoch 8/48
118/117 [==============================] - 63s - loss: 0.5301 - acc: 0.7978 - val_loss: 1.6273 - val_acc: 0.4105
Epoch 9/48
118/117 [==============================] - 63s - loss: 0.4852 - acc: 0.8166 - val_loss: 1.6836 - val_acc: 0.3858
Epoch 10/48
118/117 [==============================] - 63s - loss: 0.4357 - acc: 0.8345 - val_loss: 1.5260 - val_acc: 0.4043
Epoch 11/48
118/117 [==============================] - 62s - loss: 0.3886 - acc: 0.8596 - val_loss: 1.6285 - val_acc: 0.4012
Epoch 12/48
118/117 [==============================] - 62s - loss: 0.3622 - acc: 0.8712 - val_loss: 1.6640 - val_acc: 0.3889
Epoch 13/48
118/117 [==============================] - 62s - loss: 0.3209 - acc: 0.8879 - val_loss: 1.6548 - val_acc: 0.4136
Epoch 14/48
118/117 [==============================] - 62s - loss: 0.3019 - acc: 0.8912 - val_loss: 1.6846 - val_acc: 0.4352
Epoch 15/48
118/117 [==============================] - 62s - loss: 0.2701 - acc: 0.9069 - val_loss: 1.7094 - val_acc: 0.4228
Epoch 16/48
118/117 [==============================] - 62s - loss: 0.2458 - acc: 0.9174 - val_loss: 1.6392 - val_acc: 0.4753
Epoch 17/48
118/117 [==============================] - 62s - loss: 0.2236 - acc: 0.9235 - val_loss: 1.7241 - val_acc: 0.4228
Epoch 18/48
118/117 [==============================] - 62s - loss: 0.1945 - acc: 0.9394 - val_loss: 1.7575 - val_acc: 0.4537
Epoch 19/48
118/117 [==============================] - 62s - loss: 0.1916 - acc: 0.9396 - val_loss: 1.7809 - val_acc: 0.4290
Epoch 20/48
118/117 [==============================] - 62s - loss: 0.1666 - acc: 0.9457 - val_loss: 1.6831 - val_acc: 0.4537
Epoch 21/48
118/117 [==============================] - 63s - loss: 0.1614 - acc: 0.9476 - val_loss: 1.8463 - val_acc: 0.4259
Epoch 22/48
118/117 [==============================] - 62s - loss: 0.1324 - acc: 0.9603 - val_loss: 1.9625 - val_acc: 0.4352
Epoch 23/48
118/117 [==============================] - 63s - loss: 0.1235 - acc: 0.9648 - val_loss: 1.8510 - val_acc: 0.4444
Epoch 24/48
118/117 [==============================] - 62s - loss: 0.1263 - acc: 0.9613 - val_loss: 1.8896 - val_acc: 0.4290
Epoch 25/48
118/117 [==============================] - 63s - loss: 0.1244 - acc: 0.9595 - val_loss: 1.9483 - val_acc: 0.4383
Epoch 26/48
118/117 [==============================] - 63s - loss: 0.1006 - acc: 0.9698 - val_loss: 1.9485 - val_acc: 0.4444
Epoch 27/48
118/117 [==============================] - 63s - loss: 0.0969 - acc: 0.9727 - val_loss: 1.9763 - val_acc: 0.4506
Epoch 28/48
118/117 [==============================] - 63s - loss: 0.0962 - acc: 0.9711 - val_loss: 1.9766 - val_acc: 0.4691
Epoch 29/48
 76/117 [==================>...........] - ETA: 21s - loss: 0.0691 - acc: 0.9827